In [39]:
import pandas as pd
import numpy as np

import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode

import ipywidgets as widgets

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
init_notebook_mode(connected=True)

In [40]:
data = pd.read_csv('Data/spotify_songs.csv')
data.drop(['track_id', 'playlist_id', 'track_album_id','track_album_name' ], axis = 1, inplace = True) 
releaseYearCol = lambda x: x['track_album_release_date'].split("-")[0]
data['releaseYear'] = data.apply(releaseYearCol, axis=1)
data.head()

,track_name,track_artist,track_popularity,track_album_release_date,playlist_name,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,releaseYear
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2019-06-14,Pop Remix,pop,dance pop,0.748,0.916,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754,2019
1,Memories - Dillon Francis Remix,Maroon 5,67,2019-12-13,Pop Remix,pop,dance pop,0.726,0.815,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600,2019
2,All the Time - Don Diablo Remix,Zara Larsson,70,2019-07-05,Pop Remix,pop,dance pop,0.675,0.931,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616,2019
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,2019-07-19,Pop Remix,pop,dance pop,0.718,0.930,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093,2019
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,2019-03-05,Pop Remix,pop,dance pop,0.650,0.833,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052,2019


In [41]:
dataSankey = data[['releaseYear', 'playlist_genre', 'playlist_subgenre', 'track_popularity']].groupby(['releaseYear','playlist_genre','playlist_subgenre']).agg({'playlist_subgenre': ['count'], 'track_popularity': ['median']}).reset_index()

In [37]:
dataSankey.tail()
dataSankey.to_csv('Data/song_group.csv', index=False)

In [43]:

data_viz_1 = pd.read_csv('Data/song_group.csv')


#Choose the five year time
data_viz_1 = data_viz_1[data_viz_1['year'] > 2015]

val_in_sanky = ['year', 'genre', 'subgenre']

dimensions = [dict(values=data_viz_1[label], label=label) for label in val_in_sanky]


color = np.zeros(len(data_viz_1), dtype='uint8')
colorscale = [[0, '#F7CF3B'], [0.33, '#F7CF3B'],
              [0.33, '#2FED68'], [0.66, '#2FED68'],
              [0.66, '#2C0C50'], [1.0, '#2C0C50']]
cmin = -0.5
cmax = 2.5


fig = go.FigureWidget(
    data=[go.Scatter(x=data_viz_1.playlist_subgenre_count, y=data_viz_1['track_popularity_median'],
                marker={'color': color, 'cmin': cmin, 'cmax': cmax,
                        'colorscale': colorscale, 'showscale': True,
                        'colorbar': {'tickvals': [0, 1, 2], 'ticktext': ['None', 'Red', 'Blue']}},
                     mode='markers'),

      go.Parcats(domain={'y': [0, 0.4]}, dimensions=dimensions,
                   line={'colorscale': colorscale, 'cmin': cmin,
                   'cmax': cmax, 'color': color, 'shape': 'hspline'})]
)

fig.update_layout(height=1200, width=1500, autosize=False,xaxis={'title': 'Count'},
                  yaxis={'title': 'Popularity', 'domain': [0.6, 1]},
                  dragmode='lasso', hovermode='closest')

color_toggle = widgets.ToggleButtons(
    options=['None', 'Red', 'Blue'],
    index=1, description='Brush Color:', disabled=False)

def update_color(trace, points, state):
    new_color = np.array(fig.data[0].marker.color)
    new_color[points.point_inds] = color_toggle.index

    with fig.batch_update():
        fig.data[0].marker.color = new_color

        fig.data[1].line.color = new_color

fig.data[0].on_selection(update_color)
fig.data[1].on_click(update_color)

widgets.VBox([color_toggle, fig])

In [11]:
cars_df.head()

,year,genre,subgenre,subgen_count,highway-mpg
0,2015,edm,big room,146,24
1,2015,edm,electro house,45,41
2,2015,edm,pop edm,151,30
3,2015,edm,progressive electro house,120,38
4,2015,latin,latin hip hop,102,38
